<a href="https://colab.research.google.com/github/scottmsoh/ref_ML/blob/main/NLP_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
"""
header = 0 은 파일의 첫 번째 줄에 열 이름이 있음을 나타내며
delimiter = \t 는 필드가 탭으로 구분되는 것을 의미한다.
quoting = 3은 쌍따옴표를 무시하도록 한다.
"""
# QUOTE_MINIMAL (0), QUOTE_ALL (1),
# QUOTE_NONNUMERIC (2) or QUOTE_NONE (3).

# 레이블인 sentiment 가 있는 학습 데이터
# train = pd.read_csv('data/labeledTrainData.tsv',
#                     header=0, delimiter='\t', quoting=3)
# # 레이블이 없는 테스트 데이터
# test = pd.read_csv('data/testData.tsv',
#                    header=0, delimiter='\t', quoting=3)

# 레이블인 sentiment 가 있는 학습 데이터
train = pd.read_csv('https://github.com/corazzon/KaggleStruggle/raw/master/word2vec-nlp-tutorial/data/labeledTrainData.tsv', delimiter='\t', quoting=3)
# 레이블이 없는 테스트 데이터
test = pd.read_csv('https://github.com/corazzon/KaggleStruggle/raw/master/word2vec-nlp-tutorial/data/testData.tsv', delimiter='\t', quoting=3)

In [5]:
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [6]:
test.head()

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


In [7]:
print(train.shape)
print(test.shape)

(25000, 3)
(25000, 2)


In [8]:
# 긍정과 부정 리뷰가 어떻게 들어있는지 카운트 해본다.
# 긍정과 부정리뷰가 각각 동일하게 12,500개씩 들어있다.
train['sentiment'].value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0    id        25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      25000 non-null  object
 1   review  25000 non-null  object
dtypes: object(2)
memory usage: 390.8+ KB


In [11]:
# 전처리 모듈 가져오기
!wget https://github.com/corazzon/KaggleStruggle/raw/master/word2vec-nlp-tutorial/KaggleWord2VecUtility.py

--2023-12-21 19:04:58--  https://github.com/corazzon/KaggleStruggle/raw/master/word2vec-nlp-tutorial/KaggleWord2VecUtility.py
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/corazzon/KaggleStruggle/master/word2vec-nlp-tutorial/KaggleWord2VecUtility.py [following]
--2023-12-21 19:04:58--  https://raw.githubusercontent.com/corazzon/KaggleStruggle/master/word2vec-nlp-tutorial/KaggleWord2VecUtility.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3250 (3.2K) [text/plain]
Saving to: ‘KaggleWord2VecUtility.py’

KaggleWord2VecUtili 100%[===================>]   3.17K  --.-KB/s    in 0s      

2023-12-

In [12]:
# Necessary required tools
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [13]:
!wget https://raw.githubusercontent.com/corazzon/KaggleStruggle/master/word2vec-nlp-tutorial/KaggleWord2VecUtility.py

--2023-12-21 19:05:01--  https://raw.githubusercontent.com/corazzon/KaggleStruggle/master/word2vec-nlp-tutorial/KaggleWord2VecUtility.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3250 (3.2K) [text/plain]
Saving to: ‘KaggleWord2VecUtility.py.1’

KaggleWord2VecUtili 100%[===================>]   3.17K  --.-KB/s    in 0s      

2023-12-21 19:05:01 (59.8 MB/s) - ‘KaggleWord2VecUtility.py.1’ saved [3250/3250]



In [14]:
# 데이터 전처리는 튜토리얼 파트1~4까지 공통으로 사용되기 때문에 별도의 파이썬 파일로 분리했다.
# 그리고 캐글에 있는 코드를 병렬처리하도록 멀티프로세싱 코드를 추가했다.
# 하지만 여기에서는 멀티프로세싱 코드만 임포트해서 사용하고 전처리는 태그만 제거해주도록 한다.
# 그리고 WordNetLemmatizer로 레마타이징 한다.
# 음소표기법(lemmatization)은 튜토리얼 파트1에 설명 되어 있고 다음 링크에 있다.
# https://github.com/corazzon/KaggleStruggle/blob/master/word2vec-nlp-tutorial/tutorial-part-1.ipynb
from KaggleWord2VecUtility import KaggleWord2VecUtility
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def review_to_words( raw_review ):
    review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
    review_text = wordnet_lemmatizer.lemmatize(review_text)
    return review_text

In [15]:
# 학습데이터를 전처리 한다.
%time train['review_clean'] = KaggleWord2VecUtility.apply_by_multiprocessing(\
    train['review'], review_to_words, workers=4)

<ipython-input-14-5a4cf68a0acd>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
<ipython-input-14-5a4cf68a0acd>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
<ipython-input-14-5a4cf68a0acd>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
<ipython-input-14-5a4cf68a0acd>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = Beautifu

CPU times: user 175 ms, sys: 131 ms, total: 305 ms
Wall time: 9.51 s


In [16]:
# 학습데이터와 동일하게 테스트 데이터에 대해서도 전처리 한다.
%time test['review_clean'] = KaggleWord2VecUtility.apply_by_multiprocessing(\
    test['review'], review_to_words, workers=4)

<ipython-input-14-5a4cf68a0acd>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
<ipython-input-14-5a4cf68a0acd>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
<ipython-input-14-5a4cf68a0acd>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
<ipython-input-14-5a4cf68a0acd>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = Beautifu

CPU times: user 178 ms, sys: 116 ms, total: 294 ms
Wall time: 9.21 s


In [17]:
# 전처리한 학습 데이터 10개만을 불러와서 본다.
train['review_clean'][:10]

0    "With all this stuff going down at the moment ...
1    "\"The Classic War of the Worlds\" by Timothy ...
2    "The film starts with a manager (Nicholas Bell...
3    "It must be assumed that those who praised thi...
4    "Superbly trashy and wondrously unpretentious ...
5    "I dont know why people think this is such a b...
6    "This movie could have been very good, but com...
7    "I watched this video at a friend's house. I'm...
8    "A friend of mine bought this film for £1, and...
9    "This movie is full of references. Like \"Mad ...
Name: review_clean, dtype: object

In [18]:
# 전처리한 테스트 데이터 10개만을 불러와서 본다.
test['review_clean'][:10]

0    "Naturally in a film who's main themes are of ...
1    "This movie is a disaster within a disaster fi...
2    "All in all, this is a movie for kids. We saw ...
3    "Afraid of the Dark left me with the impressio...
4    "A very accurate depiction of small time mob l...
5    "...as valuable as King Tut's tomb! (OK, maybe...
6    "This has to be one of the biggest misfires ev...
7    "This is one of those movies I watched, and wo...
8    "The worst movie i've seen in years (and i've ...
9    "Five medical students (Kevin Bacon, David Lab...
Name: review_clean, dtype: object

In [19]:
# X_train과 X_test에 리뷰 데이터를 담아주고 이 데이터를 TF-IDF를 통해 임베딩(벡터화)해본다.
X_train = train['review_clean']
X_test = test['review_clean']

In [20]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from nltk.corpus import words

vectorizer = CountVectorizer(analyzer = 'word',
                             lowercase = True,
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = 'english',
                             min_df = 2, # 토큰이 나타날 최소 문서 개수로 오타나 자주 나오지 않는 특수한 전문용어 제거에 좋다.
                             ngram_range=(1, 3),
                             vocabulary = set(words.words()), # nltk의 words를 사용하거나 문서 자체의 사전을 만들거나 선택한다.
                             max_features = 90000
                            )
vectorizer

CountVectorizer(max_features=90000, min_df=2, ngram_range=(1, 3),
                stop_words='english',
                vocabulary={'A', 'Aani', 'Aaron', 'Aaronic', 'Aaronical',
                            'Aaronite', 'Aaronitic', 'Aaru', 'Ab', 'Ababdeh',
                            'Ababua', 'Abadite', 'Abama', 'Abanic', 'Abantes',
                            'Abarambo', 'Abaris', 'Abasgi', 'Abassin', 'Abatua',
                            'Abba', 'Abbadide', 'Abbasside', 'Abbie', 'Abby',
                            'Abderian', 'Abderite', 'Abdiel', 'Abdominales',
                            'Abe', ...})

In [21]:
pipeline = Pipeline([
    ('vect', vectorizer),
    ('tfidf', TfidfTransformer(smooth_idf = False)),
])
pipeline

Pipeline(steps=[('vect',
                 CountVectorizer(max_features=90000, min_df=2,
                                 ngram_range=(1, 3), stop_words='english',
                                 vocabulary={'A', 'Aani', 'Aaron', 'Aaronic',
                                             'Aaronical', 'Aaronite',
                                             'Aaronitic', 'Aaru', 'Ab',
                                             'Ababdeh', 'Ababua', 'Abadite',
                                             'Abama', 'Abanic', 'Abantes',
                                             'Abarambo', 'Abaris', 'Abasgi',
                                             'Abassin', 'Abatua', 'Abba',
                                             'Abbadide', 'Abbasside', 'Abbie',
                                             'Abby', 'Abderian', 'Abderite',
                                             'Abdiel', 'Abdominales', 'Abe', ...})),
                ('tfidf', TfidfTransformer(smooth_idf=False))])

In [22]:
%time X_train_tfidf_vector = pipeline.fit_transform(X_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:1380: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


CPU times: user 6.87 s, sys: 50.9 ms, total: 6.93 s
Wall time: 7.2 s


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:1682: RuntimeWarning: divide by zero encountered in divide
  idf = np.log(n_samples / df) + 1


In [23]:
vocab = vectorizer.get_feature_names_out()
print(len(vocab))
vocab[:10]

235892


array(['A', 'Aani', 'Aaron', 'Aaronic', 'Aaronical', 'Aaronite',
       'Aaronitic', 'Aaru', 'Ab', 'Ababdeh'], dtype=object)

In [24]:
%time X_test_tfidf_vector = pipeline.fit_transform(X_test)

CPU times: user 8.26 s, sys: 43.7 ms, total: 8.3 s
Wall time: 9.12 s


In [25]:
import numpy as np
dist = np.sum(X_train_tfidf_vector, axis=0)

for tag, count in zip(vocab, dist):
    print(count, tag)

pd.DataFrame(dist, columns=vocab)

[[0. 0. 0. ... 0. 0. 0.]] A


,A,Aani,Aaron,Aaronic,Aaronical,Aaronite,Aaronitic,Aaru,Ab,Ababdeh,...,zymotechnical,zymotechnics,zymotechny,zymotic,zymotically,zymotize,zymotoxic,zymurgy,zythem,zythum
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤포레스트 분류기를 사용
forest = RandomForestClassifier(
    n_estimators = 100, n_jobs = -1, random_state=2018)
forest

RandomForestClassifier(n_jobs=-1, random_state=2018)

In [27]:
%time forest = forest.fit(X_train_tfidf_vector, train['sentiment'])

CPU times: user 3min 20s, sys: 595 ms, total: 3min 21s
Wall time: 2min 5s


In [28]:
## Cross validation verification

In [29]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [30]:
k_fold = KFold(n_splits=5, shuffle=True, random_state=2018)

%time score = np.mean(cross_val_score(\
    forest, X_train_tfidf_vector, \
    train['sentiment'], cv=k_fold, scoring='roc_auc', n_jobs=-1))

CPU times: user 2.69 s, sys: 625 ms, total: 3.32 s
Wall time: 8min 22s


In [31]:
# 0.92149
'{:,.5f}'.format(score)

'0.92051'

In [32]:
%time result = forest.predict(X_test_tfidf_vector)

CPU times: user 2.51 s, sys: 11.8 ms, total: 2.52 s
Wall time: 3.12 s


In [33]:
result[:10]

array([1, 0, 0, 0, 1, 1, 0, 1, 0, 1])

In [34]:
output = pd.DataFrame(data={'id':test['id'], 'sentiment':result})
output.head()

,id,sentiment
0,"""12311_10""",1
1,"""8348_2""",0
2,"""5828_4""",0
3,"""7186_2""",0
4,"""12128_7""",1


In [35]:
output_sentiment = output['sentiment'].value_counts()
print(output_sentiment[0] - output_sentiment[1])
output_sentiment

-378


1    12689
0    12311
Name: sentiment, dtype: int64

In [36]:
# data 폴더가 없다면 생성합니다.
%mkdir data

In [37]:
output.to_csv('data/tutorial_4_tfidf_{0:.5f}.csv'.format(score), index=False, quoting=3)

In [38]:
# 그런데 점수가 너무 낮게 나옴
# local 0.92149 kaggle 0.84392
# stop_words = 'english' vocabulary = set(words.words()), smooth_idf = False
468/578

0.8096885813148789

In [39]:
## XGboost algorithm

In [40]:
# XGBClassifier 불러오기
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators=100,learning_rate=0.1)

In [41]:
# 정답 레이블을 y_train 변수에 담아 사용합니다.
y_train = train['sentiment']

In [42]:
# XGBClassifier 로 학습합니다.
xgb.fit(X_test_tfidf_vector, y_train, verbose=True)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [43]:
# predict로 예측합니다.
result = xgb.predict(X_test_tfidf_vector)
result[:5]

array([1, 1, 0, 0, 1])

In [44]:
result = result.astype(int)

In [45]:
result

array([1, 1, 0, ..., 0, 1, 1])

In [46]:
output = pd.DataFrame(data={'id':test['id'], 'sentiment':result})
output.head()

,id,sentiment
0,"""12311_10""",1
1,"""8348_2""",1
2,"""5828_4""",0
3,"""7186_2""",0
4,"""12128_7""",1


In [47]:
output_sentiment = output['sentiment'].value_counts()
output_sentiment

1    14240
0    10760
Name: sentiment, dtype: int64

In [48]:
print(output_sentiment[0] - output_sentiment[1])
output_sentiment

-3480


1    14240
0    10760
Name: sentiment, dtype: int64